<a href="https://colab.research.google.com/github/Normal-case/DeepLearning/blob/master/Cat_Dop_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
TRAIN_DIR = './drive/My Drive/인공지능 큐레이터/Deep Learning/data/cats_and_dogs_small/cats_and_dogs_small/train/'
TEST_DIR = './drive/My Drive/인공지능 큐레이터/Deep Learning/data/cats_and_dogs_small/cats_and_dogs_small/test/'

In [3]:
IM_WIDTH = 220
IM_HEIGHT = 220

In [4]:
train_datagen = ImageDataGenerator(
    rescale = 1./255, # 이미지의 값에 1/255 곱함
    rotation_range = 180, # 회전 최대 180도
    width_shift_range = 0.2, # 좌우 이동 최대 이미지 가로사이즈 20%
    height_shift_range = 0.2, # 상하 이동 최대 이미지 세로사이즈 20%
    horizontal_flip = True, # 좌우 반전 실행
    vertical_flip = True # 상하 반전 실행
)

In [5]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size = (IM_WIDTH, IM_HEIGHT),
    batch_size = 16,
    class_mode = 'binary',
    shuffle = True
)

Found 2000 images belonging to 2 classes.


In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [7]:
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size = (IM_WIDTH, IM_HEIGHT),
    batch_size = 10,
    class_mode = 'binary',
    shuffle = False
)

Found 1000 images belonging to 2 classes.


In [8]:
train_generator.n

2000

In [9]:
train_generator.batch_size

16

In [10]:
test_generator.n

1000

In [11]:
img, label = train_generator.next()
img.shape

(16, 220, 220, 3)

In [12]:
label.shape

(16,)

In [13]:
label

array([1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1.],
      dtype=float32)

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

In [15]:
model = Sequential()
# MaxPooling의 경우 strides의 경우 2가 default 값
# Conv2D의 경우 strides의 경우 1이 default 값
model.add(Conv2D(32, kernel_size = (3,3), input_shape = (220, 220, 3), activation = 'relu', padding = 'SAME'))
model.add(MaxPooling2D(pool_size = 2, padding = 'SAME'))
model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu', padding = 'SAME'))
model.add(MaxPooling2D(pool_size=2, padding = 'SAME'))
model.add(Conv2D(128, kernel_size=(3,3), activation = 'relu', padding = 'SAME'))
model.add(MaxPooling2D(pool_size=2, padding='SAME'))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 220, 220, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 110, 110, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 55, 55, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 55, 55, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0

In [16]:
model.compile(loss = 'binary_crossentropy', optimizer=Adam(lr=0.00001), metrics=['acc'])

In [17]:
model.fit(train_generator, epochs=30)

Epoch 1/30
125/125 [==============================] - 761s 6s/step - loss: 0.7008 - acc: 0.5235
Epoch 2/30
125/125 [==============================] - 27s 219ms/step - loss: 0.6890 - acc: 0.5360
Epoch 3/30
125/125 [==============================] - 27s 217ms/step - loss: 0.6860 - acc: 0.5485
Epoch 4/30
125/125 [==============================] - 27s 218ms/step - loss: 0.6801 - acc: 0.5505
Epoch 5/30
125/125 [==============================] - 28s 221ms/step - loss: 0.6707 - acc: 0.5765
Epoch 6/30
125/125 [==============================] - 28s 221ms/step - loss: 0.6646 - acc: 0.5820
Epoch 7/30
125/125 [==============================] - 28s 221ms/step - loss: 0.6649 - acc: 0.5765
Epoch 8/30
125/125 [==============================] - 28s 223ms/step - loss: 0.6564 - acc: 0.5855
Epoch 9/30
125/125 [==============================] - 28s 221ms/step - loss: 0.6533 - acc: 0.5885
Epoch 10/30
125/125 [==============================] - 28s 224ms/step - loss: 0.6565 - acc: 0.5785
Epoch 11/30
125/125 [

In [19]:
total_acc = 0
for index in range(100):
  X_test, y_test = test_generator.next()
  predict = model.predict(X_test)
  predict01 = np.where(predict > 0.5, 1, 0)
  predict02 = predict01.flatten()
  predict02 = (predict02 == y_test)
  accuracy = np.sum(predict02) / len(predict02)
  total_acc += accuracy

print('정확도:', total_acc / 100)

정확도: 0.6659999999999997
